#### Feature extraction

In [1]:
import numpy as np
import pandas as pd

df = pd.read_parquet("data/processed/df_for_agg.parquet")

In [2]:
df.head()

,customer_id,amount,gender,tr_description,day,time_category,mcc_embeddings
0,39026145,-2245.92,1.0,Оплата услуги. Банкоматы СБ РФ,0,рабочее время,"[-0.549412, 0.15420678, 0.6769444, -0.28600964..."
1,39026145,56147.89,1.0,Взнос наличных через АТМ (в своем тер.банке),1,рабочее время,"[0.29221445, -0.72320914, 0.47806624, -0.94505..."
2,39026145,-56147.89,1.0,Списание с карты по операции “перевода с карты...,1,рабочее время,"[-0.09898794, -0.3049472, -0.5152246, -0.52015..."
3,39026145,-1392.47,1.0,Покупка. POS ТУ СБ РФ,1,рабочее время,"[1.0021925, -0.20207441, -0.19583933, -1.30638..."
4,39026145,-920.83,1.0,Покупка. POS ТУ СБ РФ,2,рабочее время,"[1.0021925, -0.20207441, -0.19583933, -1.30638..."


In [3]:
df.dtypes

customer_id         int64
amount            float64
gender            float64
tr_description     object
day                 int64
time_category      object
mcc_embeddings     object
dtype: object

In [5]:
# --- 1) mean, min, max для трат (amount < 0)
df_spending = (
    df[df["amount"] < 0]
    .groupby("customer_id")["amount"]
    .agg(spending_mean="mean", spending_min="min", spending_max="max")
)

In [6]:
# --- 2) mean, min, max для зачислений (amount > 0)
df_deposit = (
    df[df["amount"] > 0]
    .groupby("customer_id")["amount"]
    .agg(deposit_mean="mean", deposit_min="min", deposit_max="max")
)

In [7]:
# --- 3) Среднее количество трат в день
spending_counts = (
    df[df["amount"] < 0]
    .groupby(["customer_id", "day"])
    .size()
    .reset_index(name="spending_count")
)
mean_spending_per_day = (
    spending_counts.groupby("customer_id")["spending_count"]
    .mean()
    .rename("mean_spending_per_day")
)

# --- 4) Среднее количество зачислений в день
deposit_counts = (
    df[df["amount"] > 0]
    .groupby(["customer_id", "day"])
    .size()
    .reset_index(name="deposit_count")
)
mean_deposit_per_day = (
    deposit_counts.groupby("customer_id")["deposit_count"]
    .mean()
    .rename("mean_deposit_per_day")
)

In [14]:
# --- 5) Агрегация embeddings по mean, std, max, min


def agg_embeddings(embeddings_list):
    emb_stack = np.stack(embeddings_list)
    return pd.Series(
        {
            "mean": emb_stack.mean(axis=0),
            "std": emb_stack.std(axis=0),
            "max": emb_stack.max(axis=0),
            "min": emb_stack.min(axis=0),
        }
    )


emb_agg = df.groupby("customer_id")["mcc_embeddings"].apply(
    lambda x: agg_embeddings(list(x))
)

# Преобразуем MultiIndex Series в DataFrame с колонками 'mean', 'std', 'max', 'min'
emb_features = emb_agg.unstack(level=1)

emb_mean = pd.DataFrame(
    emb_features["mean"].tolist(), index=emb_features.index
).add_prefix("emb_mean_")
emb_std = pd.DataFrame(
    emb_features["std"].tolist(), index=emb_features.index
).add_prefix("emb_std_")
emb_max = pd.DataFrame(
    emb_features["max"].tolist(), index=emb_features.index
).add_prefix("emb_max_")
emb_min = pd.DataFrame(
    emb_features["min"].tolist(), index=emb_features.index
).add_prefix("emb_min_")

emb_final = pd.concat([emb_mean, emb_std, emb_max, emb_min], axis=1)

In [16]:
emb_final.head()

,emb_mean_0,emb_mean_1,emb_mean_2,emb_mean_3,emb_mean_4,emb_mean_5,emb_mean_6,emb_mean_7,emb_mean_8,emb_mean_9,...,emb_min_302,emb_min_303,emb_min_304,emb_min_305,emb_min_306,emb_min_307,emb_min_308,emb_min_309,emb_min_310,emb_min_311
customer_id,,,,,,,,,,,,,,,,,,,,,
6815,-0.032929,-0.217995,0.453178,-0.581541,-0.028932,0.085508,-0.145927,-0.356707,-0.042187,-0.346182,...,0.053765,-0.239183,-0.743711,-0.803491,0.078074,-1.026487,-0.558471,-0.124872,-0.055579,-1.131777
22899,0.149913,-0.429121,0.334172,-0.726870,0.248812,-0.078295,-0.084983,-0.378629,0.124977,-0.367490,...,0.053765,-0.239183,-0.904732,-0.803490,0.217985,-1.026487,-0.375683,-0.505133,-0.055579,-1.505959
27914,-0.167673,-0.237874,0.547387,-0.584086,0.051212,0.033050,-0.293402,-0.543837,0.090679,-0.400982,...,0.053765,-0.239183,-0.595353,-0.803490,0.217985,-0.253887,-0.244771,-0.505133,-0.055579,-1.099298
28753,0.193729,-0.286436,0.114204,-0.624525,0.031440,-0.124175,0.162931,-0.086635,-0.348350,-0.423511,...,-0.465984,-0.239183,-1.050710,-0.904946,-0.526403,-1.102767,-0.756501,-0.505133,-0.055579,-1.505959
31385,0.068887,-0.146941,0.394259,-0.643706,-0.108395,0.089215,-0.068467,-0.255397,-0.278627,-0.324644,...,0.053765,-0.239183,-1.050710,-0.803490,0.073497,-1.026487,-0.578459,-0.505133,-0.055579,-1.505959


In [15]:
# --- 6) Frequency encoding для tr_description + топ-3

desc_freq = df["tr_description"].value_counts(normalize=True)
df["tr_desc_freq"] = df["tr_description"].map(desc_freq).fillna(0)

desc_agg = (
    df.groupby(["customer_id", "tr_description"])["tr_desc_freq"]
    .sum()
    .unstack(fill_value=0)
)
top3_tr = desc_agg.apply(lambda x: x.nlargest(3).index.tolist(), axis=1).to_frame(
    "top3_tr"
)
top3_tr[["top1_tr", "top2_tr", "top3_tr"]] = pd.DataFrame(
    top3_tr["top3_tr"].tolist(), index=top3_tr.index
)
top3_tr = top3_tr.drop(columns="top3_tr")

In [17]:
# --- 7) Frequency encoding для time_category + топ-2

time_freq = df["time_category"].value_counts(normalize=True)
df["time_cat_freq"] = df["time_category"].map(time_freq).fillna(0)

time_agg = (
    df.groupby(["customer_id", "time_category"])["time_cat_freq"]
    .sum()
    .unstack(fill_value=0)
)
top2_time = time_agg.apply(lambda x: x.nlargest(2).index.tolist(), axis=1).to_frame(
    "top2_time"
)
top2_time[["top1_time", "top2_time"]] = pd.DataFrame(
    top2_time["top2_time"].tolist(), index=top2_time.index
)
top2_time = top2_time.drop(columns="top2_time")

In [18]:
# --- 8) Общее количество транзакций

total_txn = df.groupby("customer_id").size().rename("total_txn")

# --- 9) Количество трат и зачислений
txn_spending_count = (
    df[df["amount"] < 0].groupby("customer_id").size().rename("spending_txn_count")
)
txn_deposit_count = (
    df[df["amount"] > 0].groupby("customer_id").size().rename("deposit_txn_count")
)

# --- 10)Доля трат к общему количеству транзакций
spending_ratio = (txn_spending_count / total_txn).rename("spending_ratio").fillna(0)

# --- 11) Средний и медианный amount для трат и зачислений
spending_stats = (
    df[df["amount"] < 0]
    .groupby("customer_id")["amount"]
    .agg(spending_median="median", spending_std="std")
)
deposit_stats = (
    df[df["amount"] > 0]
    .groupby("customer_id")["amount"]
    .agg(deposit_median="median", deposit_std="std")
)

# --- 12) Первый и последний день транзакции
first_day = df.groupby("customer_id")["day"].min().rename("first_day")
last_day = df.groupby("customer_id")["day"].max().rename("last_day")
active_days_span = (last_day - first_day).rename("active_days_span")

# --- 13) Количество уникальных tr_description и time_category
unique_tr_desc = (
    df.groupby("customer_id")["tr_description"].nunique().rename("unique_tr_desc")
)
unique_time_cat = (
    df.groupby("customer_id")["time_category"].nunique().rename("unique_time_cat")
)

# --- 14) Отношение средней суммы списаний к средней сумме зачислений
mean_spending = df[df["amount"] < 0].groupby("customer_id")["amount"].mean()
mean_deposit = df[df["amount"] > 0].groupby("customer_id")["amount"].mean()
mean_spending_deposit_ratio = (
    (mean_spending.abs() / mean_deposit)
    .rename("mean_spending_deposit_ratio")
    .replace(np.inf, np.nan)
    .fillna(0)
)

In [ ]:
# --- Финальное объединение всех признаков в один датафрейм
df_agg = pd.concat(
    [
        df_spending,
        df_deposit,
        mean_spending_per_day,
        mean_deposit_per_day,
        emb_final,
        top3_tr,
        top2_time,
        total_txn,
        txn_spending_count,
        txn_deposit_count,
        spending_ratio,
        spending_stats,
        deposit_stats,
        first_day,
        last_day,
        active_days_span,
        unique_tr_desc,
        unique_time_cat,
        mean_spending_deposit_ratio,
    ],
    axis=1,
).reset_index()

# Объединение с таргетом (gender)
gender_df = df[["customer_id", "gender"]].drop_duplicates(subset=["customer_id"])
df_agg = df_agg.merge(gender_df, on="customer_id", how="left")

# df_agg — готовый датасет с агрегатами для модели

In [21]:
df_agg.head()

,customer_id,spending_mean,spending_min,spending_max,deposit_mean,deposit_min,deposit_max,mean_spending_per_day,mean_deposit_per_day,emb_mean_0,...,spending_std,deposit_median,deposit_std,first_day,last_day,active_days_span,unique_tr_desc,unique_time_cat,mean_spending_deposit_ratio,gender
0,6815,-17981.854409,-224591.58,-22.46,451092.183333,224.59,2470507.35,1.410256,1.000000,-0.032929,...,29652.262014,50533.105,990028.956041,10,454,444,6,4,0.039863,NaN
1,22899,-38847.708675,-868607.92,-88.04,85572.397794,88.04,875907.15,1.765957,1.307692,0.149913,...,102071.385485,33688.740,141569.144017,1,456,455,21,4,0.453975,1.0
2,27914,-44564.559474,-462658.65,-1122.96,11710.848571,1122.96,89836.63,1.357143,1.129032,-0.167673,...,121931.651195,5614.790,20519.704519,13,451,438,7,4,3.805408,1.0
3,28753,-296121.687828,-3244539.76,-2245.92,914148.179400,7860.71,3099363.77,1.648649,1.428571,0.193729,...,602533.653083,700905.395,805859.113805,0,452,452,15,4,0.323932,0.0
4,31385,-10033.152135,-364366.15,-16.84,54405.845652,2245.92,199886.50,2.326531,1.210526,0.068887,...,27343.958680,41549.440,53114.569478,0,436,436,17,4,0.184413,0.0


In [22]:
df_agg.to_parquet("data/processed/df_agg.parquet", engine="pyarrow")